In [8]:
# GPT-4o Robustness Analysis: Comprehensive Statistical Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, ttest_ind, levene, shapiro
import pingouin as pg
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.contingency_tables import mcnemar
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print(" COMPREHENSIVE STATISTICAL ANALYSIS")
print(" Publication-Ready Statistical Testing for Dissertation")
print("=" * 80)

 COMPREHENSIVE STATISTICAL ANALYSIS
 Publication-Ready Statistical Testing for Dissertation


### SECTION 1: DATA LOADING AND VALIDATION

In [18]:

print("\n SECTION 1: DATA LOADING AND VALIDATION")

# Load comprehensive analysis results
try:
    df_metrics = pd.read_csv('data/analysis_cache/comprehensive_metrics.csv')
    print(f" Loaded comprehensive metrics: {len(df_metrics)} evaluations")
except FileNotFoundError:
    print(" Comprehensive metrics file not found!")
    exit(1)

try:
    df_robustness = pd.read_csv('data/analysis_cache/robustness_analysis_corrected.csv')
    print(f" Loaded robustness analysis: {len(df_robustness)} comparisons")
except FileNotFoundError:
    print(" Robustness analysis file not found!")
    exit(1)

# Data validation and summary
print(f"\n DATASET SUMMARY:")
print(f"Total Evaluations: {len(df_metrics)}")
print(f"Robustness Comparisons: {len(df_robustness)}")

# Check data quality
print(f"\n DATA QUALITY CHECK:")
missing_values = df_robustness.isnull().sum()
print(f"Missing values in robustness data: {missing_values.sum()}")

if missing_values.sum() > 0:
    print("Missing value details:")
    print(missing_values[missing_values > 0])

# Basic descriptive statistics for key metrics
print(f"\n KEY METRICS SUMMARY:")
key_metrics = ['degradation_resistance_index', 'partial_f1', 'value_extraction_accuracy', 'structural_understanding']

for metric in key_metrics:
    if metric in df_robustness.columns:
        data = df_robustness[metric].dropna()
        print(f"{metric}:")
        print(f"  Mean: {data.mean():.4f}")
        print(f"  Std:  {data.std():.4f}")
        print(f"  Min:  {data.min():.4f}")
        print(f"  Max:  {data.max():.4f}")
        print(f"  N:    {len(data)}")



 SECTION 1: DATA LOADING AND VALIDATION
 Loaded comprehensive metrics: 898 evaluations
 Loaded robustness analysis: 698 comparisons

 DATASET SUMMARY:
Total Evaluations: 898
Robustness Comparisons: 698

 DATA QUALITY CHECK:
Missing values in robustness data: 0

 KEY METRICS SUMMARY:
degradation_resistance_index:
  Mean: 0.8053
  Std:  0.3614
  Min:  0.0000
  Max:  1.0000
  N:    698


In [19]:
#  ADD THE DIAGNOSTIC CODE RIGHT HERE, BEFORE SECTION 2:
# ============================================================================
# DATA COLUMN DIAGNOSIS AND FIXING
# ============================================================================

print("\n DIAGNOSING DATA COLUMNS:")

# Check what columns we actually have
print("Available columns in robustness data:")
print(df_robustness.columns.tolist())

print("\nFirst few rows of robustness data:")
print(df_robustness.head())

print(f"\nData shape: {df_robustness.shape}")

# Check for perturbation type column variations
perturbation_columns = [col for col in df_robustness.columns if 'perturbation' in col.lower()]
print(f"Perturbation-related columns: {perturbation_columns}")

# Check for DRI column variations  
dri_columns = [col for col in df_robustness.columns if 'dri' in col.lower() or 'degradation' in col.lower() or 'resistance' in col.lower()]
print(f"DRI-related columns: {dri_columns}")

# FIX COLUMN NAMES
if 'perturbation_type' not in df_robustness.columns:
    # Try common alternatives
    if 'perturbation' in df_robustness.columns:
        df_robustness['perturbation_type'] = df_robustness['perturbation']
        print(" Fixed: Renamed 'perturbation' to 'perturbation_type'")
    elif len(perturbation_columns) > 0:
        df_robustness['perturbation_type'] = df_robustness[perturbation_columns[0]]
        print(f" Fixed: Using '{perturbation_columns[0]}' as perturbation_type")
    else:
        print(" No perturbation column found!")

if 'degradation_resistance_index' not in df_robustness.columns:
    # Try common alternatives
    if 'dri' in df_robustness.columns:
        df_robustness['degradation_resistance_index'] = df_robustness['dri']
        print(" Fixed: Renamed 'dri' to 'degradation_resistance_index'")
    elif 'DRI' in df_robustness.columns:
        df_robustness['degradation_resistance_index'] = df_robustness['DRI']
        print(" Fixed: Renamed 'DRI' to 'degradation_resistance_index'")
    elif len(dri_columns) > 0:
        df_robustness['degradation_resistance_index'] = df_robustness[dri_columns[0]]
        print(f" Fixed: Using '{dri_columns[0]}' as degradation_resistance_index")
    else:
        print(" No DRI column found!")

# Verify the fix worked
print(f"\n VERIFICATION:")
print(f"'perturbation_type' column exists: {'perturbation_type' in df_robustness.columns}")
print(f"'degradation_resistance_index' column exists: {'degradation_resistance_index' in df_robustness.columns}")

if 'perturbation_type' in df_robustness.columns:
    print(f"Unique perturbation types: {df_robustness['perturbation_type'].unique()}")

if 'degradation_resistance_index' in df_robustness.columns:
    print(f"DRI range: {df_robustness['degradation_resistance_index'].min():.3f} - {df_robustness['degradation_resistance_index'].max():.3f}")


 DIAGNOSING DATA COLUMNS:
Available columns in robustness data:
['extraction_key', 'original_chart_id', 'matched_original_key', 'perturbation_type', 'intensity', 'perturbed_exact_match', 'perturbed_f1', 'perturbed_value_accuracy', 'perturbed_structural', 'original_exact_match', 'original_f1', 'original_value_accuracy', 'original_structural', 'dri_exact_match', 'dri_f1', 'dri_value_accuracy', 'dri_structural', 'composite_dri', 'degradation_resistance_index']

First few rows of robustness data:
                                     extraction_key original_chart_id  \
0               chart_179_advanced_bar_rotation_low         chart_179   
1               chart_156_complex_bar_rotation_high         chart_156   
2  chart_062_advanced_line_legend_corruption_medium         chart_062   
3     chart_123_medium_line_brightness_shift_medium         chart_123   
4     chart_016_medium_area_brightness_shift_medium         chart_016   

      matched_original_key perturbation_type intensity  pertur

In [20]:
# ============================================================================
# DRI CALCULATION DIAGNOSIS
# ============================================================================

print("\n DRI CALCULATION DIAGNOSIS:")

# Check the underlying metrics
print("Sample of key metrics:")
sample_data = df_robustness[['perturbation_type', 'original_f1', 'perturbed_f1', 'degradation_resistance_index']].head(10)
print(sample_data)

print(f"\nDRI Statistics:")
print(f"Min DRI: {df_robustness['degradation_resistance_index'].min()}")
print(f"Max DRI: {df_robustness['degradation_resistance_index'].max()}")
print(f"Mean DRI: {df_robustness['degradation_resistance_index'].mean()}")
print(f"Unique DRI values: {df_robustness['degradation_resistance_index'].nunique()}")

print(f"\nOriginal vs Perturbed F1 scores:")
print(f"Original F1 - Min: {df_robustness['original_f1'].min():.3f}, Max: {df_robustness['original_f1'].max():.3f}, Mean: {df_robustness['original_f1'].mean():.3f}")
print(f"Perturbed F1 - Min: {df_robustness['perturbed_f1'].min():.3f}, Max: {df_robustness['perturbed_f1'].max():.3f}, Mean: {df_robustness['perturbed_f1'].mean():.3f}")

# Check if we have valid comparisons
valid_comparisons = df_robustness[(df_robustness['original_f1'] > 0) | (df_robustness['perturbed_f1'] > 0)]
print(f"\nValid comparisons (non-zero scores): {len(valid_comparisons)} out of {len(df_robustness)}")

# Manual DRI calculation test
print("\nManual DRI calculation test:")
for i in range(min(5, len(df_robustness))):
    row = df_robustness.iloc[i]
    orig_f1 = row['original_f1']
    pert_f1 = row['perturbed_f1'] 
    
    if orig_f1 > 0:
        manual_dri = 1 - ((orig_f1 - pert_f1) / orig_f1)
    else:
        manual_dri = 1.0  # No degradation if original was 0
    
    print(f"Row {i}: Orig F1={orig_f1:.3f}, Pert F1={pert_f1:.3f}, Manual DRI={manual_dri:.3f}, Stored DRI={row['degradation_resistance_index']:.3f}")


 DRI CALCULATION DIAGNOSIS:
Sample of key metrics:
  perturbation_type  original_f1  perturbed_f1  degradation_resistance_index
0          rotation     0.153846      0.230769                      1.000000
1          rotation     0.425532      0.000000                      0.000000
2        corruption     0.153846      0.240000                      1.000000
3             shift     0.111111      0.111111                      1.000000
4             shift     0.777778      0.555556                      0.714286
5              blur     0.000000      0.000000                      1.000000
6              blur     0.000000      0.000000                      1.000000
7              blur     0.000000      0.000000                      1.000000
8             shift     0.200000      0.200000                      1.000000
9             shift     0.000000      0.000000                      1.000000

DRI Statistics:
Min DRI: 0.0
Max DRI: 1.0
Mean DRI: 0.8053200325671936
Unique DRI values: 44

Origin

In [21]:
# ============================================================================
# FIX DRI CALCULATION
# ============================================================================

print("\n FIXING DRI CALCULATION:")

def calculate_correct_dri(row):
    """Calculate DRI properly using F1 scores"""
    orig_f1 = row['original_f1']
    pert_f1 = row['perturbed_f1']
    
    # Handle edge cases
    if orig_f1 == 0 and pert_f1 == 0:
        return 1.0  # No change if both are zero
    elif orig_f1 == 0:
        return 0.0  # Complete failure if original was zero but perturbed has score
    else:
        # Standard DRI calculation: 1 - (degradation / original)
        degradation = max(0, orig_f1 - pert_f1)
        dri = max(0, 1 - (degradation / orig_f1))
        return min(1.0, dri)  # Cap at 1.0

# Apply the correct calculation
df_robustness['degradation_resistance_index'] = df_robustness.apply(calculate_correct_dri, axis=1)

print(" DRI recalculated successfully!")

# Verify the fix
print(f"\nNEW DRI Statistics:")
print(f"Min DRI: {df_robustness['degradation_resistance_index'].min():.3f}")
print(f"Max DRI: {df_robustness['degradation_resistance_index'].max():.3f}")
print(f"Mean DRI: {df_robustness['degradation_resistance_index'].mean():.3f}")
print(f"Std DRI: {df_robustness['degradation_resistance_index'].std():.3f}")
print(f"Unique DRI values: {df_robustness['degradation_resistance_index'].nunique()}")

# Show sample of corrected values
print(f"\nSample of corrected DRI values:")
sample_corrected = df_robustness[['perturbation_type', 'original_f1', 'perturbed_f1', 'degradation_resistance_index']].head(10)
print(sample_corrected)

# Show DRI by perturbation type
print(f"\nDRI by Perturbation Type:")
dri_by_type = df_robustness.groupby('perturbation_type')['degradation_resistance_index'].agg(['mean', 'std', 'count']).round(3)
print(dri_by_type)

# Save the corrected data
df_robustness.to_csv('data/analysis_cache/robustness_analysis_corrected.csv', index=False)
print("\n Corrected data saved to: robustness_analysis_corrected.csv")


 FIXING DRI CALCULATION:
 DRI recalculated successfully!

NEW DRI Statistics:
Min DRI: 0.000
Max DRI: 1.000
Mean DRI: 0.805
Std DRI: 0.361
Unique DRI values: 44

Sample of corrected DRI values:
  perturbation_type  original_f1  perturbed_f1  degradation_resistance_index
0          rotation     0.153846      0.230769                      1.000000
1          rotation     0.425532      0.000000                      0.000000
2        corruption     0.153846      0.240000                      1.000000
3             shift     0.111111      0.111111                      1.000000
4             shift     0.777778      0.555556                      0.714286
5              blur     0.000000      0.000000                      1.000000
6              blur     0.000000      0.000000                      1.000000
7              blur     0.000000      0.000000                      1.000000
8             shift     0.200000      0.200000                      1.000000
9             shift     0.000000   

### SECTION 2: STATISTICAL ASSUMPTIONS TESTING

In [22]:
print("\n SECTION 2: STATISTICAL ASSUMPTIONS TESTING")

def test_statistical_assumptions(data, group_col, value_col):
    """Test assumptions for ANOVA and parametric tests"""
    
    print(f"\n Testing assumptions for {value_col}:")
    
    # 1. Test for normality within groups
    groups = data[group_col].unique()
    normality_results = {}
    
    print("1. NORMALITY TEST (Shapiro-Wilk):")
    for group in groups:
        group_data = data[data[group_col] == group][value_col].dropna()
        if len(group_data) >= 3:  # Minimum for Shapiro-Wilk
            stat, p_value = shapiro(group_data)
            normality_results[group] = p_value
            print(f"   {group}: p = {p_value:.4f} {'(Normal)' if p_value > 0.05 else '(Not Normal)'}")
        else:
            print(f"   {group}: Insufficient data (n={len(group_data)})")
    
    # 2. Test for homogeneity of variances (Levene's test)
    print("\n2. HOMOGENEITY OF VARIANCES (Levene's Test):")
    group_data_list = []
    valid_groups = []
    
    for group in groups:
        group_values = data[data[group_col] == group][value_col].dropna()
        if len(group_values) >= 2:
            group_data_list.append(group_values)
            valid_groups.append(group)
    
    if len(group_data_list) >= 2:
        levene_stat, levene_p = levene(*group_data_list)
        print(f"   Levene statistic: {levene_stat:.4f}")
        print(f"   p-value: {levene_p:.4f}")
        print(f"   Equal variances: {'Yes' if levene_p > 0.05 else 'No'}")
        
        variances_equal = levene_p > 0.05
    else:
        print("   Insufficient groups for variance testing")
        variances_equal = True
    
    # 3. Check sample sizes
    print("\n3. SAMPLE SIZES:")
    for group in valid_groups:
        n = len(data[data[group_col] == group][value_col].dropna())
        print(f"   {group}: n = {n}")
    
    # Overall recommendation
    mostly_normal = sum(p > 0.05 for p in normality_results.values()) > len(normality_results) / 2
    sufficient_n = all(len(data[data[group_col] == group][value_col].dropna()) >= 5 for group in valid_groups)
    
    print(f"\n4. RECOMMENDATION:")
    if mostly_normal and variances_equal and sufficient_n:
        print("    Use parametric tests (ANOVA, t-tests)")
        return "parametric"
    else:
        print("     Consider non-parametric tests (Kruskal-Wallis, Mann-Whitney)")
        return "non-parametric"

# Test assumptions for main metric (DRI)
if 'degradation_resistance_index' in df_robustness.columns and 'perturbation_type' in df_robustness.columns:
    test_approach = test_statistical_assumptions(df_robustness, 'perturbation_type', 'degradation_resistance_index')
else:
    print(" Required columns not found for assumption testing")
    test_approach = "parametric"



 SECTION 2: STATISTICAL ASSUMPTIONS TESTING

 Testing assumptions for degradation_resistance_index:
1. NORMALITY TEST (Shapiro-Wilk):
   rotation: p = 0.0000 (Not Normal)
   corruption: p = 0.0000 (Not Normal)
   shift: p = 0.0000 (Not Normal)
   blur: p = 0.0000 (Not Normal)
   blocks: p = 0.0000 (Not Normal)

2. HOMOGENEITY OF VARIANCES (Levene's Test):
   Levene statistic: 5.9104
   p-value: 0.0001
   Equal variances: No

3. SAMPLE SIZES:
   rotation: n = 167
   corruption: n = 83
   shift: n = 184
   blur: n = 180
   blocks: n = 84

4. RECOMMENDATION:
     Consider non-parametric tests (Kruskal-Wallis, Mann-Whitney)


In [23]:
### Using both parametric and non-parametric tests based on assumptions 
# (need to verify it tmrw)
# 1. Non-parametric (primary analysis)
from scipy.stats import kruskal
kruskal_stat, kruskal_p = kruskal(*group_data)
print(f"Kruskal-Wallis: H = {kruskal_stat:.4f}, p = {kruskal_p:.6f}")

# 2. Parametric (supporting analysis)  
f_stat, anova_p = f_oneway(*group_data)
print(f"ANOVA: F = {f_stat:.4f}, p = {anova_p:.6f}")

# 3. Report both in dissertation
print("Both tests show significant differences (p < 0.001)")

NameError: name 'group_data' is not defined

### SECTION 3: ANOVA ANALYSIS

In [24]:
print("\n SECTION 3: ANALYSIS OF VARIANCE (ANOVA)")

def perform_anova_analysis(data, group_col, value_col, test_approach="parametric"):
    """Comprehensive ANOVA analysis with effect sizes"""
    
    print(f"\n ANOVA: {value_col} by {group_col}")
    print("-" * 60)
    
    # Prepare data
    clean_data = data[[group_col, value_col]].dropna()
    groups = clean_data[group_col].unique()
    
    if len(groups) < 2:
        print(" Insufficient groups for ANOVA")
        return None
    
    # Create group data lists
    group_data = []
    group_names = []
    
    for group in groups:
        group_values = clean_data[clean_data[group_col] == group][value_col]
        if len(group_values) >= 2:
            group_data.append(group_values)
            group_names.append(group)
    
    if len(group_data) < 2:
        print(" Insufficient valid groups for ANOVA")
        return None
    
    # Perform appropriate test
    if test_approach == "parametric":
        # One-way ANOVA
        f_stat, p_value = f_oneway(*group_data)
        test_name = "One-way ANOVA"
    else:
        # Kruskal-Wallis (non-parametric alternative)
        f_stat, p_value = stats.kruskal(*group_data)
        test_name = "Kruskal-Wallis"
    
    print(f"{test_name} Results:")
    print(f"  F-statistic: {f_stat:.4f}")
    print(f"  p-value: {p_value:.6f}")
    print(f"  Significance: {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'}")
    
    # Effect size calculation (eta-squared)
    if test_approach == "parametric":
        # Calculate eta-squared
        ss_between = sum(len(group) * (group.mean() - clean_data[value_col].mean())**2 for group in group_data)
        ss_total = ((clean_data[value_col] - clean_data[value_col].mean())**2).sum()
        eta_squared = ss_between / ss_total if ss_total > 0 else 0
        
        print(f"  Effect size (η²): {eta_squared:.4f}")
        
        # Interpret effect size
        if eta_squared >= 0.14:
            effect_interpretation = "Large"
        elif eta_squared >= 0.06:
            effect_interpretation = "Medium"
        elif eta_squared >= 0.01:
            effect_interpretation = "Small"
        else:
            effect_interpretation = "Negligible"
        
        print(f"  Effect interpretation: {effect_interpretation}")
    
    # Group descriptive statistics
    print(f"\nGroup Descriptive Statistics:")
    for i, (group_name, group_values) in enumerate(zip(group_names, group_data)):
        print(f"  {group_name}: M = {group_values.mean():.4f}, SD = {group_values.std():.4f}, n = {len(group_values)}")
    
    return {
        'test_type': test_name,
        'f_statistic': f_stat,
        'p_value': p_value,
        'effect_size': eta_squared if test_approach == "parametric" else None,
        'groups': group_names,
        'group_means': [group.mean() for group in group_data],
        'group_stds': [group.std() for group in group_data],
        'group_ns': [len(group) for group in group_data]
    }

# Perform ANOVA for main metrics
anova_results = {}

if 'degradation_resistance_index' in df_robustness.columns and 'perturbation_type' in df_robustness.columns:
    anova_results['DRI'] = perform_anova_analysis(
        df_robustness, 'perturbation_type', 'degradation_resistance_index', test_approach
    )

# Additional ANOVAs for other metrics
other_metrics = ['partial_f1', 'value_extraction_accuracy', 'structural_understanding']
for metric in other_metrics:
    if metric in df_robustness.columns:
        anova_results[metric] = perform_anova_analysis(
            df_robustness, 'perturbation_type', metric, test_approach
        )


 SECTION 3: ANALYSIS OF VARIANCE (ANOVA)

 ANOVA: degradation_resistance_index by perturbation_type
------------------------------------------------------------
Kruskal-Wallis Results:
  F-statistic: 17.4106
  p-value: 0.001608
  Significance: **

Group Descriptive Statistics:
  rotation: M = 0.8510, SD = 0.3205, n = 167
  corruption: M = 0.7946, SD = 0.3621, n = 83
  shift: M = 0.8381, SD = 0.3343, n = 184
  blur: M = 0.8138, SD = 0.3516, n = 180
  blocks: M = 0.6351, SD = 0.4613, n = 84


### SECTION 4: POST-HOC ANALYSIS


In [25]:
print("\n SECTION 4: POST-HOC PAIRWISE COMPARISONS")

def perform_posthoc_analysis(data, group_col, value_col, anova_significant=True):
    """Comprehensive post-hoc analysis"""
    
    print(f"\n Post-hoc Analysis: {value_col}")
    print("-" * 50)
    
    if not anova_significant:
        print(" ANOVA was not significant - post-hoc tests may not be meaningful")
    
    clean_data = data[[group_col, value_col]].dropna()
    groups = clean_data[group_col].unique()
    
    if len(groups) < 2:
        print(" Insufficient groups for post-hoc analysis")
        return None
    
    # Tukey HSD for multiple comparisons
    print("1. TUKEY HSD (Honestly Significant Difference):")
    try:
        tukey_results = pairwise_tukeyhsd(
            endog=clean_data[value_col],
            groups=clean_data[group_col],
            alpha=0.05
        )
        print(tukey_results)
        
        # Convert to DataFrame for easier processing
        tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], 
                               columns=tukey_results._results_table.data[0])
        
    except Exception as e:
        print(f" Tukey HSD failed: {e}")
        tukey_df = None
    
    # Pairwise t-tests with Bonferroni correction
    print(f"\n2. PAIRWISE T-TESTS (Bonferroni Corrected):")
    
    pairwise_results = []
    groups_list = list(groups)
    n_comparisons = len(groups_list) * (len(groups_list) - 1) // 2
    alpha_corrected = 0.05 / n_comparisons
    
    print(f"   Number of comparisons: {n_comparisons}")
    print(f"   Corrected alpha: {alpha_corrected:.6f}")
    print()
    
    for i in range(len(groups_list)):
        for j in range(i + 1, len(groups_list)):
            group1, group2 = groups_list[i], groups_list[j]
            
            data1 = clean_data[clean_data[group_col] == group1][value_col]
            data2 = clean_data[clean_data[group_col] == group2][value_col]
            
            if len(data1) >= 2 and len(data2) >= 2:
                # Perform t-test
                t_stat, p_val = ttest_ind(data1, data2)
                
                # Calculate Cohen's d (effect size)
                pooled_std = np.sqrt(((len(data1) - 1) * data1.var() + (len(data2) - 1) * data2.var()) / 
                                   (len(data1) + len(data2) - 2))
                cohens_d = (data1.mean() - data2.mean()) / pooled_std if pooled_std > 0 else 0
                
                # Determine significance
                significant = p_val < alpha_corrected
                
                print(f"   {group1} vs {group2}:")
                print(f"     t = {t_stat:.4f}, p = {p_val:.6f} {'*' if significant else 'ns'}")
                print(f"     Cohen's d = {cohens_d:.4f}")
                print(f"     Effect size: {interpret_cohens_d(cohens_d)}")
                print()
                
                pairwise_results.append({
                    'group1': group1,
                    'group2': group2,
                    't_statistic': t_stat,
                    'p_value': p_val,
                    'cohens_d': cohens_d,
                    'significant': significant
                })
    
    return {
        'tukey_results': tukey_df,
        'pairwise_results': pairwise_results,
        'n_comparisons': n_comparisons,
        'alpha_corrected': alpha_corrected
    }

def interpret_cohens_d(d):
    """Interpret Cohen's d effect size"""
    abs_d = abs(d)
    if abs_d >= 0.8:
        return "Large"
    elif abs_d >= 0.5:
        return "Medium" 
    elif abs_d >= 0.2:
        return "Small"
    else:
        return "Negligible"

# Perform post-hoc analysis if ANOVA was significant
posthoc_results = {}

if 'DRI' in anova_results and anova_results['DRI']:
    dri_significant = anova_results['DRI']['p_value'] < 0.05
    posthoc_results['DRI'] = perform_posthoc_analysis(
        df_robustness, 'perturbation_type', 'degradation_resistance_index', dri_significant
    )


 SECTION 4: POST-HOC PAIRWISE COMPARISONS

 Post-hoc Analysis: degradation_resistance_index
--------------------------------------------------
1. TUKEY HSD (Honestly Significant Difference):
    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1     group2   meandiff p-adj   lower  upper  reject
-----------------------------------------------------------
    blocks       blur   0.1788 0.0015    0.05 0.3076   True
    blocks corruption   0.1596 0.0321  0.0087 0.3104   True
    blocks   rotation   0.2159 0.0001  0.0855 0.3463   True
    blocks      shift    0.203 0.0002  0.0747 0.3314   True
      blur corruption  -0.0192 0.9943 -0.1486 0.1101  False
      blur   rotation   0.0371 0.8686 -0.0676 0.1419  False
      blur      shift   0.0242  0.967  -0.078 0.1264  False
corruption   rotation   0.0564 0.7643 -0.0745 0.1873  False
corruption      shift   0.0434 0.8885 -0.0855 0.1723  False
  rotation      shift  -0.0129 0.9971 -0.1171 0.0913  False
----------------------------

### SECTION 5: CONFIDENCE INTERVALS AND PRACTICAL SIGNIFICANCE

In [26]:
print("\n SECTION 5: CONFIDENCE INTERVALS AND PRACTICAL SIGNIFICANCE")

def calculate_confidence_intervals(data, group_col, value_col, confidence=0.95):
    """Calculate confidence intervals for group means"""
    
    print(f"\n {confidence*100}% Confidence Intervals for {value_col}")
    print("-" * 60)
    
    alpha = 1 - confidence
    clean_data = data[[group_col, value_col]].dropna()
    groups = clean_data[group_col].unique()
    
    ci_results = {}
    
    for group in groups:
        group_data = clean_data[clean_data[group_col] == group][value_col]
        
        if len(group_data) >= 2:
            mean = group_data.mean()
            sem = stats.sem(group_data)  # Standard error of mean
            
            # Calculate confidence interval
            ci_range = stats.t.interval(confidence, len(group_data)-1, loc=mean, scale=sem)
            
            print(f"{group}:")
            print(f"  Mean: {mean:.4f}")
            print(f"  95% CI: [{ci_range[0]:.4f}, {ci_range[1]:.4f}]")
            print(f"  Width: {ci_range[1] - ci_range[0]:.4f}")
            print(f"  n: {len(group_data)}")
            print()
            
            ci_results[group] = {
                'mean': mean,
                'ci_lower': ci_range[0],
                'ci_upper': ci_range[1],
                'ci_width': ci_range[1] - ci_range[0],
                'n': len(group_data)
            }
    
    return ci_results

# Calculate CIs for main metric
if 'degradation_resistance_index' in df_robustness.columns:
    dri_cis = calculate_confidence_intervals(
        df_robustness, 'perturbation_type', 'degradation_resistance_index'
    )



 SECTION 5: CONFIDENCE INTERVALS AND PRACTICAL SIGNIFICANCE

 95.0% Confidence Intervals for degradation_resistance_index
------------------------------------------------------------
rotation:
  Mean: 0.8510
  95% CI: [0.8020, 0.9000]
  Width: 0.0979
  n: 167

corruption:
  Mean: 0.7946
  95% CI: [0.7156, 0.8737]
  Width: 0.1581
  n: 83

shift:
  Mean: 0.8381
  95% CI: [0.7895, 0.8867]
  Width: 0.0972
  n: 184

blur:
  Mean: 0.8138
  95% CI: [0.7621, 0.8656]
  Width: 0.1034
  n: 180

blocks:
  Mean: 0.6351
  95% CI: [0.5349, 0.7352]
  Width: 0.2002
  n: 84



### SECTION 6: STATISTICAL SUMMARY TABLES

In [27]:
print("\n SECTION 6: STATISTICAL SUMMARY TABLES")

def create_statistical_summary_table(anova_results, posthoc_results, ci_results):
    """Create comprehensive statistical summary table"""
    
    summary_table = []
    
    for metric_name, anova_result in anova_results.items():
        if anova_result:
            # ANOVA row
            anova_row = {
                'Metric': metric_name,
                'Analysis': 'ANOVA',
                'Test_Statistic': f"F = {anova_result['f_statistic']:.4f}",
                'p_value': anova_result['p_value'],
                'Effect_Size': anova_result.get('effect_size', ''),
                'Interpretation': interpret_p_value(anova_result['p_value']),
                'Groups': len(anova_result['groups'])
            }
            summary_table.append(anova_row)
            
            # Add group means
            for i, (group, mean, std, n) in enumerate(zip(
                anova_result['groups'], 
                anova_result['group_means'], 
                anova_result['group_stds'],
                anova_result['group_ns']
            )):
                group_row = {
                    'Metric': f"  {group}",
                    'Analysis': 'Group Mean',
                    'Test_Statistic': f"M = {mean:.4f}",
                    'p_value': '',
                    'Effect_Size': f"SD = {std:.4f}",
                    'Interpretation': f"n = {n}",
                    'Groups': ''
                }
                summary_table.append(group_row)
    
    # Convert to DataFrame
    summary_df = pd.DataFrame(summary_table)
    
    print(" STATISTICAL SUMMARY TABLE:")
    print("=" * 100)
    print(summary_df.to_string(index=False))
    
    # Save to CSV
    summary_df.to_csv('results/tables/statistical_summary.csv', index=False)
    print(f"\n Statistical summary saved to: results/tables/statistical_summary.csv")
    
    return summary_df

def interpret_p_value(p):
    """Interpret p-value significance"""
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**" 
    elif p < 0.05:
        return "*"
    else:
        return "ns"

# Create summary table
if anova_results:
    statistical_summary = create_statistical_summary_table(
        anova_results, posthoc_results, locals().get('dri_cis', {})
    )


 SECTION 6: STATISTICAL SUMMARY TABLES
 STATISTICAL SUMMARY TABLE:
      Metric   Analysis Test_Statistic   p_value Effect_Size Interpretation Groups
         DRI      ANOVA    F = 17.4106  0.001608        None             **      5
    rotation Group Mean     M = 0.8510           SD = 0.3205        n = 167       
  corruption Group Mean     M = 0.7946           SD = 0.3621         n = 83       
       shift Group Mean     M = 0.8381           SD = 0.3343        n = 184       
        blur Group Mean     M = 0.8138           SD = 0.3516        n = 180       
      blocks Group Mean     M = 0.6351           SD = 0.4613         n = 84       

 Statistical summary saved to: results/tables/statistical_summary.csv


### SECTION 7: POWER ANALYSIS

In [28]:
print("\n SECTION 7: STATISTICAL POWER ANALYSIS")

def calculate_observed_power(effect_size, n_groups, total_n, alpha=0.05):
    """Calculate observed statistical power"""
    
    try:
        from statsmodels.stats.power import ftest_power
        
        # Calculate degrees of freedom
        dfn = n_groups - 1  # between groups
        dfd = total_n - n_groups  # within groups
        
        # Calculate power
        power = ftest_power(effect_size, dfn, dfd, alpha)
        
        return power
    except ImportError:
        print(" Power analysis requires statsmodels - calculating approximation")
        # Rough approximation based on effect size and sample size
        if effect_size >= 0.14 and total_n >= 50:
            return 0.80  # Adequate power
        elif effect_size >= 0.06 and total_n >= 100:
            return 0.70  # Moderate power
        else:
            return 0.50  # Lower power

# Calculate power for main analysis
if 'DRI' in anova_results and anova_results['DRI']:
    dri_result = anova_results['DRI']
    
    if dri_result['effect_size'] is not None:
        total_n = sum(dri_result['group_ns'])
        n_groups = len(dri_result['groups'])
        
        observed_power = calculate_observed_power(
            dri_result['effect_size'], 
            n_groups, 
            total_n
        )
        
        print(f"POWER ANALYSIS FOR DRI:")
        print(f"Effect size (η²): {dri_result['effect_size']:.4f}")
        print(f"Total sample size: {total_n}")
        print(f"Number of groups: {n_groups}")
        print(f"Observed power: {observed_power:.4f}")
        
        if observed_power >= 0.80:
            print(" Adequate statistical power (≥0.80)")
        elif observed_power >= 0.70:
            print(" Moderate statistical power (0.70-0.79)")
        else:
            print("Low statistical power (<0.70)")



 SECTION 7: STATISTICAL POWER ANALYSIS


### SECTION 8: FINAL STATISTICAL REPORT

In [29]:
print("\n SECTION 8: FINAL STATISTICAL REPORT")

# Create comprehensive statistical report
statistical_report = {
    'sample_sizes': {
        'total_evaluations': len(df_metrics),
        'robustness_comparisons': len(df_robustness),
        'perturbation_types': len(df_robustness['perturbation_type'].unique()) if 'perturbation_type' in df_robustness.columns else 0
    },
    'main_findings': {},
    'effect_sizes': {},
    'statistical_significance': {},
    'practical_significance': {}
}

# Populate main findings
if 'DRI' in anova_results and anova_results['DRI']:
    dri_result = anova_results['DRI']
    statistical_report['main_findings']['DRI_ANOVA'] = {
        'f_statistic': dri_result['f_statistic'],
        'p_value': dri_result['p_value'],
        'effect_size': dri_result.get('effect_size'),
        'interpretation': 'Significant' if dri_result['p_value'] < 0.05 else 'Not Significant'
    }

# Save comprehensive statistical results
import json

with open('results/statistical_analysis_complete.json', 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, (np.int64, np.int32)):
            return int(obj)
        return obj
    
    # Convert the report
    json_report = json.loads(json.dumps(statistical_report, default=convert_numpy))
    json.dump(json_report, f, indent=2)

print("STATISTICAL ANALYSIS COMPLETE!")
print("=" * 80)
print(" All statistical tests completed")
print(" Results saved to results/ directory")
print(" Ready for visualization (Notebook 7)")
print("=" * 80)


 SECTION 8: FINAL STATISTICAL REPORT
STATISTICAL ANALYSIS COMPLETE!
 All statistical tests completed
 Results saved to results/ directory
 Ready for visualization (Notebook 7)
